In [1]:
import json
import nltk
import pandas as pd
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from wordcloud import WordCloud
from nltk.corpus import stopwords
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import hmean
from scipy.stats import norm
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, concatenate, Activation
from keras.layers import Flatten, Conv1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Model
#nltk.download('stopwords')

Using TensorFlow backend.


In [4]:
#Read Train Dataset
tweets=pd.read_csv("D:/Thesis/Code/Data/train_data.csv")

In [5]:
tweets.head()

,sentiment,text
0,1,Two places I'd invest all my money if I could:...
1,1,Awesome! Google driverless cars will help the ...
2,0,If Google maps can't keep up with road constru...
3,0,Autonomous cars seem way overhyped given the t...
4,2,Just saw Google self-driving car on I-34. It w...


In [6]:
#Data Cleaning
tok = WordPunctTokenizer()
#stop_words = set(stopwords.words('english')) 
pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
pat3 = r'RT @[A-Za-z0-9_]+'
pat4 = r'RT @[A-Za-z0-9_ ] RT+'
#combined_pat = r'|'.join((pat1, pat2,pat3))
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')
rt_pattern= re.compile(pat3,flags=re.IGNORECASE)
def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    if rt_pattern.match(bom_removed):
        rt_stripped=re.sub(pat3,'',bom_removed)
        stripped = re.sub(combined_pat, '', rt_stripped)
    else:
        stripped = re.sub(combined_pat, '', bom_removed)
        stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    #tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [7]:
x = len(tweets.index)
nums = [0,x]
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%525 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_tweet_texts.append(tweet_cleaner_updated(tweets['text'][i]))
    clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])

Cleaning and parsing the tweets...

Tweets 525 of 5250 has been processed
Tweets 1050 of 5250 has been processed
Tweets 1575 of 5250 has been processed
Tweets 2100 of 5250 has been processed
Tweets 2625 of 5250 has been processed
Tweets 3150 of 5250 has been processed
Tweets 3675 of 5250 has been processed
Tweets 4200 of 5250 has been processed
Tweets 4725 of 5250 has been processed
Tweets 5250 of 5250 has been processed


In [8]:
clean_df['target'] = tweets.sentiment

In [ ]:
clean_df.head()

In [9]:
x = clean_df.text
y = clean_df.target
SEED = 5
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.3, random_state=SEED)

In [10]:
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [11]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [12]:
#Pure Distributed Bag of Words
cores = multiprocessing.cpu_count()
model_ug_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha
    
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs
  
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 100)

clf = LogisticRegression()
clf.fit(train_vecs_dbow, y_train)
clf.score(validation_vecs_dbow, y_validation)

C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2629579.17it/s]
W0813 15:36:39.027686 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 5261671.68it/s]
W0813 15:36:39.190251 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2632723.10it/s]
W0813 15:36:39.354812 16860 base_any2vec.py:1386] under 10 

W0813 15:36:43.906968 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6696315120711563

In [13]:
#Pure Distributed Memory Mean
cores = multiprocessing.cpu_count()
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha
    
train_vecs_dmm = get_vectors(model_ug_dmm, x_train, 100)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_validation, 100)

clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)
clf.score(validation_vecs_dmm, y_validation)

C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 1754868.98it/s]
W0813 15:36:46.176894 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2632093.71it/s]
W0813 15:36:46.418249 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 5264187.43it/s]
W0813 15:36:46.692630 16860 base_any2vec.py:1386] under 10 

W0813 15:36:54.015976 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6531130876747141

In [14]:
#DBOW+DMM
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

train_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_train, 200)
validation_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_validation, 200)

clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm, y_train)
clf.score(validation_vecs_dbow_dmm, y_validation)

C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6670902160101652

In [15]:
#Separate Word2Vec
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2,
                         workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2634297.88it/s]


In [16]:
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]),
                        total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2,
                       workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]),
                      total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

def get_w2v_mean(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += np.append(model_ug_cbow[word],model_ug_sg[word]).reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec
  
def get_w2v_sum(tweet, size):
    vec = np.zeros(size).reshape((1, size))
    for word in tweet.split():
        try:
            vec += np.append(model_ug_cbow[word],model_ug_sg[word]).reshape((1, size))
        except KeyError:
            continue
    return vec
  
train_vecs_cbowsg_mean = scale(np.concatenate([get_w2v_mean(z, 200) for z in x_train]))
validation_vecs_cbowsg_mean = scale(np.concatenate([get_w2v_mean(z, 200) for z in x_validation]))
clf = LogisticRegression()
clf.fit(train_vecs_cbowsg_mean, y_train)
print(clf.score(validation_vecs_cbowsg_mean, y_validation))

train_vecs_cbowsg_sum = scale(np.concatenate([get_w2v_sum(z, 200) for z in x_train]))
validation_vecs_cbowsg_sum = scale(np.concatenate([get_w2v_sum(z, 200) for z in x_validation]))
clf = LogisticRegression()
clf.fit(train_vecs_cbowsg_sum, y_train)
print(clf.score(validation_vecs_cbowsg_sum, y_validation))

100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2632093.71it/s]
W0813 15:37:01.637448 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2647600.82it/s]
W0813 15:37:01.686319 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 1754170.00it/s]
W0813 15:37:01.729203 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 5091351.68it/s]
W0813 15:37:01.772123 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a

W0813 15:37:03.000974 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2605928.52it/s]
W0813 15:37:03.049813 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 2577258.43it/s]
W0813 15:37:03.101674 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|█████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 5266705.57it/s]
W0813 15:37:03.150576 16860 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
100%|███████████████████████████████████████████████████████████████████

C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6721728081321474


C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6645489199491741


In [17]:
#w2v general
def get_w2v_general(tweet, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count
        return vec
    elif aggregation == 'sum':
        return vec

In [18]:
#Word vectors extracted from Doc2Vec models with custom weighting
cvec = CountVectorizer(max_features=100000)
cvec.fit(x_train)

neg_train = x_train[y_train == 0]
pos_train = x_train[y_train == 1]
neu_train = x_train[y_train == 2]
neg_doc_matrix = cvec.transform(neg_train)
pos_doc_matrix = cvec.transform(pos_train)
neu_doc_matrix = cvec.transform(neu_train)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neu_tf = np.sum(neu_doc_matrix,axis=0)

def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())

neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
neu = np.squeeze(np.asarray(neu_tf))
term_freq_df2 = pd.DataFrame([neg,pos,neu],columns=cvec.get_feature_names()).transpose()
term_freq_df2.columns = ['negative', 'positive','neutral']
term_freq_df2['total'] = term_freq_df2['negative'] + term_freq_df2['positive'] + term_freq_df2['neutral']
term_freq_df2['pos_rate'] = term_freq_df2['positive'] * 1./term_freq_df2['total']
term_freq_df2['pos_freq_pct'] = term_freq_df2['positive'] * 1./term_freq_df2['positive'].sum()
term_freq_df2['pos_rate_normcdf'] = normcdf(term_freq_df2['pos_rate'])
term_freq_df2['pos_freq_pct_normcdf'] = normcdf(term_freq_df2['pos_freq_pct'])
term_freq_df2['pos_normcdf_hmean'] = hmean([term_freq_df2['pos_rate_normcdf'], term_freq_df2['pos_freq_pct_normcdf']])
pos_hmean = term_freq_df2.pos_normcdf_hmean

w2v_pos_hmean = {}
for w in model_ug_dbow.wv.vocab.keys():
    if w in pos_hmean.keys():
        w2v_pos_hmean[w] = np.append(model_ug_dbow[w],model_ug_dmm[w]) * pos_hmean[w]
        
train_vecs_w2v_poshmean_mean = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean, 'mean') for z in x_train]))
validation_vecs_w2v_poshmean_mean = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean, 'mean') for z in x_validation]))
clf = LogisticRegression()
clf.fit(train_vecs_w2v_poshmean_mean, y_train)
print(clf.score(validation_vecs_w2v_poshmean_mean, y_validation))

train_vecs_w2v_poshmean_sum = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean, 'sum') for z in x_train]))
validation_vecs_w2v_poshmean_sum = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean, 'sum') for z in x_validation]))
clf = LogisticRegression()
clf.fit(train_vecs_w2v_poshmean_sum, y_train)
print(clf.score(validation_vecs_w2v_poshmean_sum, y_validation))

C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6747141041931385


C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6670902160101652


In [19]:
#Separately trained Word2Vec with custom weighting (Average/Sum)
w2v_pos_hmean_01 = {}
for w in model_ug_cbow.wv.vocab.keys():
    if w in pos_hmean.keys():
        w2v_pos_hmean_01[w] = np.append(model_ug_cbow[w],model_ug_sg[w]) * pos_hmean[w]
        
train_vecs_w2v_poshmean_mean_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean_01, 'mean') for z in x_train]))
validation_vecs_w2v_poshmean_mean_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean_01, 'mean') for z in x_validation]))
clf = LogisticRegression()
clf.fit(train_vecs_w2v_poshmean_mean_01, y_train)
print(clf.score(validation_vecs_w2v_poshmean_mean_01, y_validation))

train_vecs_w2v_poshmean_sum_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean_01, 'sum') for z in x_train]))
validation_vecs_w2v_poshmean_sum_01 = scale(np.concatenate([get_w2v_general(z, 200, w2v_pos_hmean_01, 'sum') for z in x_validation]))
clf = LogisticRegression()
clf.fit(train_vecs_w2v_poshmean_sum_01, y_train)
print(clf.score(validation_vecs_w2v_poshmean_sum_01, y_validation))

C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6658195679796697


C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6772554002541296


In [ ]:
model_ug_cbow.save("w2v_model_ug_cbow.word2vec")

In [ ]:
model_ug_sg.save("w2v_model_ug_sg.word2vec")

In [ ]:
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('w2v_model_ug_sg.word2vec')

In [20]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [44]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)

30

In [22]:
x_train_seq = pad_sequences(sequences, maxlen=45)

In [23]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=45)
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
x_train_seq[:5]

In [24]:
model_ptw2v = Sequential()
e = Embedding(100000, 200, input_length=45)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(512, activation='relu'))
model_ptw2v.add(Dense(10, activation='softmax'))
model_ptw2v.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=64, verbose=2)

W0813 15:37:42.636605 16860 deprecation_wrapper.py:119] From C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0813 15:37:42.651576 16860 deprecation_wrapper.py:119] From C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 15:37:42.655541 16860 deprecation_wrapper.py:119] From C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0813 15:37:42.687489 16860 deprecation_wrapper.py:119] From C:\Users\Asus FX504\.conda\envs\tensor\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0813 15:37:42.702416 16

Train on 3675 samples, validate on 787 samples
Epoch 1/5
 - 15s - loss: 0.9307 - acc: 0.6180 - val_loss: 0.8163 - val_acc: 0.6480
Epoch 2/5
 - 14s - loss: 0.5617 - acc: 0.7875 - val_loss: 0.7838 - val_acc: 0.6595
Epoch 3/5
 - 15s - loss: 0.1850 - acc: 0.9488 - val_loss: 0.9097 - val_acc: 0.6760
Epoch 4/5
 - 15s - loss: 0.0664 - acc: 0.9837 - val_loss: 0.9723 - val_acc: 0.6633
Epoch 5/5
 - 16s - loss: 0.0436 - acc: 0.9878 - val_loss: 1.0342 - val_acc: 0.6607


In [65]:
tweet_input = Input(shape=(45,), dtype='int32')

tweet_encoder = Embedding(100000, 200, input_length=45)(tweet_input)
bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(512, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(10)(merged)
output = Activation('softmax')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 45)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 45, 200)      20000000    input_9[0][0]                    
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 44, 100)      40100       embedding_9[0][0]                
__________________________________________________________________________________________________
conv1d_23 (Conv1D)              (None, 43, 100)      60100       embedding_9[0][0]                
__________________________________________________________________________________________________
conv1d_24 

In [66]:
from keras.callbacks import ModelCheckpoint

filepath="CNN_best_weights.{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, y_train, batch_size=64, epochs=10,
                     validation_data=(x_val_seq, y_validation), callbacks = [checkpoint])

Train on 3675 samples, validate on 787 samples
Epoch 1/10
3675/3675 [==============================] - ETA: 1:32 - loss: 2.3236 - acc: 0.0000e+0 - ETA: 52s - loss: 2.2378 - acc: 0.2656    - ETA: 38s - loss: 2.1641 - acc: 0.36 - ETA: 31s - loss: 2.1004 - acc: 0.41 - ETA: 27s - loss: 2.0398 - acc: 0.43 - ETA: 25s - loss: 1.9614 - acc: 0.46 - ETA: 23s - loss: 1.8842 - acc: 0.48 - ETA: 21s - loss: 1.8192 - acc: 0.49 - ETA: 20s - loss: 1.7378 - acc: 0.51 - ETA: 18s - loss: 1.6684 - acc: 0.52 - ETA: 17s - loss: 1.5987 - acc: 0.54 - ETA: 16s - loss: 1.5719 - acc: 0.54 - ETA: 15s - loss: 1.5209 - acc: 0.55 - ETA: 15s - loss: 1.4878 - acc: 0.55 - ETA: 14s - loss: 1.4625 - acc: 0.55 - ETA: 13s - loss: 1.4200 - acc: 0.56 - ETA: 13s - loss: 1.3949 - acc: 0.56 - ETA: 12s - loss: 1.3641 - acc: 0.57 - ETA: 12s - loss: 1.3349 - acc: 0.58 - ETA: 11s - loss: 1.3127 - acc: 0.58 - ETA: 11s - loss: 1.2998 - acc: 0.58 - ETA: 10s - loss: 1.2750 - acc: 0.58 - ETA: 10s - loss: 1.2568 - acc: 0.58 - ETA: 10s - l

3675/3675 [==============================] - ETA: 12s - loss: 0.2222 - acc: 0.95 - ETA: 12s - loss: 0.2437 - acc: 0.93 - ETA: 12s - loss: 0.2466 - acc: 0.94 - ETA: 12s - loss: 0.2282 - acc: 0.94 - ETA: 11s - loss: 0.2233 - acc: 0.95 - ETA: 11s - loss: 0.2099 - acc: 0.95 - ETA: 11s - loss: 0.2088 - acc: 0.94 - ETA: 11s - loss: 0.2073 - acc: 0.94 - ETA: 11s - loss: 0.2163 - acc: 0.94 - ETA: 10s - loss: 0.2301 - acc: 0.93 - ETA: 10s - loss: 0.2291 - acc: 0.93 - ETA: 10s - loss: 0.2284 - acc: 0.92 - ETA: 10s - loss: 0.2243 - acc: 0.93 - ETA: 9s - loss: 0.2283 - acc: 0.9297 - ETA: 9s - loss: 0.2262 - acc: 0.931 - ETA: 9s - loss: 0.2236 - acc: 0.934 - ETA: 9s - loss: 0.2255 - acc: 0.933 - ETA: 8s - loss: 0.2257 - acc: 0.935 - ETA: 8s - loss: 0.2231 - acc: 0.937 - ETA: 8s - loss: 0.2200 - acc: 0.939 - ETA: 8s - loss: 0.2243 - acc: 0.936 - ETA: 8s - loss: 0.2216 - acc: 0.938 - ETA: 7s - loss: 0.2212 - acc: 0.938 - ETA: 7s - loss: 0.2241 - acc: 0.936 - ETA: 7s - loss: 0.2255 - acc: 0.935 - ETA:

3675/3675 [==============================] - ETA: 12s - loss: 0.0280 - acc: 0.98 - ETA: 12s - loss: 0.0207 - acc: 0.99 - ETA: 12s - loss: 0.0162 - acc: 0.99 - ETA: 12s - loss: 0.0306 - acc: 0.99 - ETA: 12s - loss: 0.0291 - acc: 0.99 - ETA: 11s - loss: 0.0264 - acc: 0.99 - ETA: 11s - loss: 0.0317 - acc: 0.99 - ETA: 11s - loss: 0.0359 - acc: 0.99 - ETA: 11s - loss: 0.0349 - acc: 0.98 - ETA: 10s - loss: 0.0330 - acc: 0.99 - ETA: 10s - loss: 0.0319 - acc: 0.99 - ETA: 10s - loss: 0.0385 - acc: 0.98 - ETA: 10s - loss: 0.0412 - acc: 0.98 - ETA: 10s - loss: 0.0411 - acc: 0.98 - ETA: 9s - loss: 0.0400 - acc: 0.9865 - ETA: 9s - loss: 0.0380 - acc: 0.987 - ETA: 9s - loss: 0.0374 - acc: 0.988 - ETA: 9s - loss: 0.0372 - acc: 0.988 - ETA: 8s - loss: 0.0356 - acc: 0.989 - ETA: 8s - loss: 0.0341 - acc: 0.989 - ETA: 8s - loss: 0.0344 - acc: 0.988 - ETA: 8s - loss: 0.0333 - acc: 0.989 - ETA: 7s - loss: 0.0324 - acc: 0.989 - ETA: 7s - loss: 0.0314 - acc: 0.990 - ETA: 7s - loss: 0.0307 - acc: 0.990 - ETA:

3675/3675 [==============================] - ETA: 14s - loss: 0.0051 - acc: 1.00 - ETA: 13s - loss: 0.0039 - acc: 1.00 - ETA: 13s - loss: 0.0121 - acc: 0.99 - ETA: 12s - loss: 0.0101 - acc: 0.99 - ETA: 12s - loss: 0.0262 - acc: 0.99 - ETA: 12s - loss: 0.0222 - acc: 0.99 - ETA: 12s - loss: 0.0194 - acc: 0.99 - ETA: 11s - loss: 0.0229 - acc: 0.99 - ETA: 11s - loss: 0.0207 - acc: 0.99 - ETA: 11s - loss: 0.0195 - acc: 0.99 - ETA: 11s - loss: 0.0186 - acc: 0.99 - ETA: 10s - loss: 0.0226 - acc: 0.99 - ETA: 10s - loss: 0.0303 - acc: 0.99 - ETA: 10s - loss: 0.0310 - acc: 0.99 - ETA: 10s - loss: 0.0316 - acc: 0.98 - ETA: 10s - loss: 0.0312 - acc: 0.98 - ETA: 9s - loss: 0.0299 - acc: 0.9899 - ETA: 9s - loss: 0.0285 - acc: 0.990 - ETA: 9s - loss: 0.0279 - acc: 0.990 - ETA: 9s - loss: 0.0267 - acc: 0.990 - ETA: 8s - loss: 0.0259 - acc: 0.991 - ETA: 8s - loss: 0.0254 - acc: 0.991 - ETA: 8s - loss: 0.0257 - acc: 0.991 - ETA: 8s - loss: 0.0250 - acc: 0.991 - ETA: 7s - loss: 0.0251 - acc: 0.991 - ETA:

In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#tvec = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
#tvec.fit(x_train)

#x_train_tfidf = tvec.transform(x_train)
#x_test_tfidf = tvec.transform(x_test)
#lr_with_tfidf = LogisticRegression()
#lr_with_tfidf.fit(x_train_tfidf,y_train)
#lr_with_tfidf.score(x_test_tfidf,y_test)
#yhat_lr = lr_with_tfidf.predict_proba(x_test_tfidf)

In [67]:
from keras.models import load_model
loaded_CNN_model = load_model('CNN_best_weights.02-0.71.hdf5')

In [68]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=45)

In [69]:
loaded_CNN_model.evaluate(x=x_test_seq, y=y_test)

788/788 [==============================] - ETA:  - ETA:  - ETA:  - 0s 571us/step


[0.7618807530040064, 0.7017766500487546]

In [52]:
yhat_cnn = loaded_CNN_model.predict(x_test_seq)

### Test DataSet Work

In [ ]:
#For Reading Test Dataset
def read_data(tweets_data_path):
    tweets_data = []
    tweets_file = open(tweets_data_path, "r")
    line = tweets_file.readline()
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue
    data = pd.DataFrame.from_dict(tweets_data)
    return data

In [ ]:
#Read Test Dataset
tweet_data=read_data("D:/Thesis/autonomousDriveMulti.json")
tweet_data1=read_data("D:/Thesis/autonomousDrive.json")
tweet_data2=read_data("D:/Thesis/python.json")
tweets= pd.concat([tweet_data, tweet_data1,tweet_data2], ignore_index=True)

In [ ]:
#For Test Dataset Only
tweets_by_lang = tweets['lang'].value_counts()

fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('Languages', fontsize=15)
ax.set_ylabel('Number of tweets' , fontsize=15)
ax.set_title('Top 5 languages', fontsize=15, fontweight='bold')
tweets_by_lang[:5].plot(ax=ax, kind='bar')

In [ ]:
#For Test Dataset Only
tweets_by_lang = tweets['source'].value_counts()
fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('Source', fontsize=15)
ax.set_ylabel('Number of tweets' , fontsize=15)
ax.set_title('Top 5 languages', fontsize=15, fontweight='bold')
tweets_by_lang[:5].plot(ax=ax, kind='bar')

In [ ]:
#tweets_data_path = 'D:/Thesis/autonomousDrive.json'

#tweets_data = []
#tweets_file = open(tweets_data_path, "r")
#line = tweets_file.readline()
#for line in tweets_file:
#    try:
#        tweet = json.loads(line)
#        tweets_data.append(tweet)
#    except:
#        continue

In [ ]:
#tweets = pd.DataFrame.from_dict(tweets_data)
#tweets = pd.read_excel("D:\\Thesis\\Code\\tweet.xlsx")

In [ ]:
#Conversion to xlsx (optional)
writer = ExcelWriter('tweet.xlsx')
tweets.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
#For Test Dataset Only
tweets['created_at'] = pd.to_datetime(tweets.created_at)
tweets['source'] = tweets['source'].str.extract('>(.+?)<', expand=False).str.strip()
tweets['pre_clean_len'] = [len(t) for t in tweets.text]

In [ ]:
#For Test Dataset Only
fig, ax = plt.subplots(figsize=(5, 5))
plt.boxplot(tweets.pre_clean_len)
plt.show()

In [ ]:
#Test Data Clean
x = len(tweets.index)
nums = [0,x]
print("Cleaning and parsing the tweets...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%881 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_tweet_texts.append(tweet_cleaner_updated(tweets['text'][i]))
    clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])

In [ ]:
text = " ".join(message for message in tweets.text)
# Create and generate a word cloud image:
wordcloud = WordCloud(width = 2400, height = 1600).generate(text)
# Display the generated image:
plt.imshow(wordcloud, interpolation='gaussian')
plt.axis("off")
plt.show()

In [ ]:
time_len=pd.Series(data=tweets['pre_clean_len'].values, index=tweets['created_at'])
time_len.plot(figsize=(16, 4), color='r')
plt.show()

In [ ]:
#clean_df.groupby(level=0).first()

# END